# Lab 3

Author: Justin Ventura [[jventura3@gulls.salisbury.edu]]

Date: Thursday, September 17th, 2020.

## - Description -

This is where the fun begins >:)

### Import libraries which will be used and extract data:

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

a_frame = pd.read_csv("adult.data", header=None,
                      skipinitialspace=True, # this data has some extra whitespace
                      names=['age', 'workclass', 'fnlwgt', 'education',
                             'education-num', 'marital-status', 
                             'occupation', 'relationship', 'race',
                             'sex', 'capital_gain', 'capital_loss',
                             'hr_per_week', 'country', 'income'
                            ])
print(a_frame.shape)  # This outputs in form: (row, col).
a_frame.head()        # Print the first 5 entries.

(32561, 15)


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital_gain,capital_loss,hr_per_week,country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### Pivoting the data.  

#### a) Rank occupation by most likely to make an income over $50k yearly.

In [52]:
# Create a sub-frame of the adult data with just the columns 'occupation' and 'income'.
occ_income = a_frame[['occupation', 'income']]

# Create two sub-frames of the previous sub-frame corresponding to two groups:
# Group 1: all of those who make over $50K, grouped by occupation, and
# Group 2: all of those who make under $50K, grouped by occupation.
greater = occ_income[occ_income['income'] == '>50K'].groupby('occupation').count()
lesser =  occ_income[occ_income['income'] == '<=50K'].groupby('occupation').count()

In [58]:
# Print the frames
print('All groups of occupations making over $50K:')

greater['income'][0]

All groups of occupations making over $50K:


KeyError: 0

In [40]:
print('All groups of occupations making under $50K:')
lesser

All groups of occupations making under $50K:


,income
occupation,
?,1652
Adm-clerical,3263
Armed-Forces,8
Craft-repair,3170
Exec-managerial,2098
Farming-fishing,879
Handlers-cleaners,1284
Machine-op-inspct,1752
Other-service,3158


In [47]:
# Find the ratio for each occupation as a:b where a is the number of
# individuals who make over $50K, and b is the total number in that
# sample occupation.
totals = []
for pair in zip(greater['income'].to_numpy(), lesser['income'].to_numpy()):
    totals.append(sum(pair))

totals

[1843,
 3770,
 9,
 4099,
 4066,
 994,
 1370,
 2002,
 3295,
 149,
 4140,
 649,
 3650,
 928,
 1597]